In [2]:
import os 
import openai
import langchain
import warnings
import numpy as np 
from dotenv import load_dotenv, find_dotenv 

_ = load_dotenv(find_dotenv())

warnings.filterwarnings("ignore")

In [3]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader

Load the documents from the directory using the `DirectoryLoader`

In [13]:
loader = DirectoryLoader(
    path="./docs",
    glob="**/*.txt",
    loader_cls=TextLoader,
)

raw_docs = loader.load()

In [14]:
print(f"Number of docs: {len(raw_docs)}")

Number of docs: 22


In [15]:
print(raw_docs[0].page_content[:500])

Hey ,

It's finally here. The Complete Guide To Getting Started With Open Source.

It's a 3 part course (4 parts for software engineers), and will take you ~30 minutes to complete.

Open source completely transformed my life as a software engineer, and I strongly believe it can do the same for you too.

[Watch The Course for Free Here](https://www.crushingit.tech/courses/open-source/what-is-open-source-with-examples?utm_source=newsletter&utm_medium=email&utm_campaign=3-popular-job-interview-ques


Split the documents into chunks

In [16]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(raw_docs)

Create and store embeddings in the vector database (in-memory)

In [18]:
embeddings = OpenAIEmbeddings()
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

In [19]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
    return_generated_question=True,
)

In [23]:
result = qa_chain({
    "question": "What is your advice for junior developers?",
    "chat_history": []
})

print(result["answer"])

My advice for junior developers would be:

1. Focus on understanding the fundamentals of the trade rather than just acquiring specific skills. Learn about different technologies and frameworks, but also understand the problems they solve and the mental models they use.

2. Strive for excellence and aim to be better than the average developer. Learn how to build world-class solutions to problems and deliver high-quality work. Don't settle for mediocre results.

3. Take ownership of the entire software development life cycle. Your role as a software engineer goes beyond just writing code. Understand the bigger picture, think strategically, and consider aspects like solution architecture and documentation.

Remember, becoming a valuable software engineer takes time and effort. Keep learning, practicing, and seeking opportunities to grow your skills and knowledge.


In [24]:
result = qa_chain({
    "question": "What is the most important skill for a software engineer?",
    "chat_history": []
})

print(result["answer"])

The most important skill for a software engineer is problem solving.


In [25]:
result = qa_chain({
    "question": "How can I get jobs as a software engineer?",
    "chat_history": []
})

print(result["answer"])

To increase your chances of getting jobs as a software engineer, here are three important steps you can take:

1. Learn the fundamentals of the trade: It's crucial to have a strong foundation in software engineering. Make sure you understand the core concepts, programming languages, and technologies that are commonly used in the industry. This will enable you to solve problems effectively and make informed decisions when choosing technologies for projects.

2. Understand the business problems inside out: Companies value engineers who can solve their problems in a fast and cost-effective manner. Take the time to understand the business side of things, such as the market you're in and the current talent pool of the company. This will allow you to make strategic technology decisions and demonstrate your ability to provide value to the company.

3. Build a strong portfolio: In today's competitive job market, having an empty GitHub profile or portfolio won't cut it. Employers want to see ev

In [26]:
result = qa_chain({
    "question": "How can I ace job interviews?",
    "chat_history": []
})

print(result["answer"])

To ace job interviews, there are two key factors to focus on:

1. Demonstrating Competence: Showcasing your skills and qualifications is crucial. Prepare by identifying the specific requirements and responsibilities of the job you're applying for, and then highlight relevant experiences and achievements that align with those requirements. Even if you don't have professional experience, you can create your own experiences, such as working on open-source projects or personal projects, and discuss them confidently during the interview.

2. Being Likable: Remember that interviewers are human, and their impressions of you can heavily influence their decision-making. Practice good communication skills, maintain a positive attitude, and engage in friendly conversation during the interview. Building rapport and showing genuine interest in the company and the role can help make a positive impression.

By focusing on both competence and likability, you can increase your chances of acing job inte

In [27]:
print(result["source_documents"])

[Document(page_content="As a takeaway from this letter, answer these two questions:\n\n1. Am I a competent candidate in every interview ?\n2. Do my interviewers like me at the end of the interview ?\n\nYou don't have to be searching for a job to master these skills. You can master them today, and increase your chances of winning big at interviews.\n\nThank you for reading. Until next week, stay growing.\n\n             [Online version](https://frantz.mailcoach.app/webview/campaign/ab0c3d98-81ad-41b6-9a3a-ddcb70d6b92a) â€¢ [Unsubscribe](https://frantz.mailcoach.app/unsubscribe/fbba01de-5bd1-468f-a377-3337dae3438e/b774b743-76b0-4056-92ab-a7b43f5d1d6e)     Sent by [Frantz Kati](https://twitter.com/bahdcoder?utm_source=newsletter&utm_medium=email&utm_campaign=crushing-the-tech-interview-the-two-secret-formulas)", metadata={'source': 'docs\\Crushing_the_tech_interview_The_two_secret_formulas.txt'}), Document(page_content="For example.\n\nSay the job description mentioned how important autom